In [1]:
import pandas as pd
import numpy as np
import wrds
import statsmodels.api as sm
import getFamaFrenchFactors as gff

# Download CRSP stock information
conn = wrds.Connection()
crsp = conn.raw_sql("""
                      select a.permno, 
                      a.permco,
                      a.date, 
                      b.shrcd, 
                      b.exchcd, 
                      a.ret, 
                      a.vol, 
                      a.shrout, 
                      a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1965' and '12/31/1989'
                      and b.exchcd between 1 and 3
                      and b.shrcd between 10 and 11
                      """)

# Download Fama and French 3 Factors
ff3 = gff.famaFrench3Factor(frequency='m')
ff3['date']=pd.to_datetime(ff3['date_ff_factors'], format='%Y-%m-%d')
ff3['year-month']=ff3['date'].apply(lambda x:str(x.year)+'-'+str(x.month))

Enter your WRDS username [rory]:rorysrose
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [2]:
df=crsp
df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d')
df['prc']=df['prc'].apply(lambda x: abs(x))
df['year-month']=df['date'].apply(lambda x:str(x.year)+'-'+str(x.month))
df.sort_values(by=['permno', 'date'], ascending=True, inplace=True)
df=df.reset_index(drop=True)
df['logret'] = np.log(1+df['ret'])

In [3]:
lagged=6 # J-month lagged returns
holdingperiod=6 # K-month holding returns
skipmonth=1

df['momentum_{}_logreturn'.format(lagged)] =\
np.exp(df.groupby('permno')['logret'].rolling(lagged).sum().reset_index(drop=True))-1
df['holding_{}_skip_{}_logreturn'.format(holdingperiod, skipmonth)] =\
np.exp(df.groupby('permno')['logret'].rolling(
    holdingperiod).sum().reset_index(drop=True).shift(
    -holdingperiod-skipmonth))-1
df_=df.dropna().reset_index(drop=True)

In [4]:
# remove some extreme values
percentile=0.01
bottom_extreme_value=df_['momentum_6_logreturn'].quantile(percentile)
top_extreme_value=df_['momentum_6_logreturn'].quantile(1-percentile)
df_=\
df_[(df_['momentum_6_logreturn'] >= bottom_extreme_value)&\
    (df_['momentum_6_logreturn'] <= top_extreme_value)].reset_index(drop=True)

num_percentile=10
df_['momentum_{}_logreturn_rank_{}_{}'.format(lagged,\
                                              1,\
                                              num_percentile)]=\
df_.groupby('date')['momentum_{}_logreturn'.format(lagged)].transform(
    lambda x: pd.qcut(x.rank(),
                      num_percentile,
                      labels=False))+1
_df_=\
df_.groupby(['date',
             'momentum_{}_logreturn_rank_{}_{}'.format(lagged, 1, num_percentile)]
           )['holding_{}_skip_{}_logreturn'.format(holdingperiod, skipmonth)].mean().reset_index(drop=False)
_df_['holding_{}_skip_{}_logreturn'.format(holdingperiod, skipmonth)]=\
_df_['holding_{}_skip_{}_logreturn'.format(holdingperiod, skipmonth)]/6 # monthly return

### Merge with Fama French 3 Factors

In [124]:
_df_portfolio=\
_df_.pivot(index='date',
           columns='momentum_{}_logreturn_rank_{}_{}'.format(lagged, 1, num_percentile),
           values='holding_{}_skip_{}_logreturn'.format(holdingperiod, skipmonth)).reset_index(drop=False)
_df_portfolio['year-month']=\
_df_portfolio['date'].apply(lambda x: str(x.year)+'-'+str(x.month))

In [125]:
table = pd.merge(_df_portfolio,
                 ff3.drop(columns = ['date', 'date_ff_factors']),
                 how='left', on=['year-month'])

In [126]:
table['10-1']=table[10]-table[1]

In [127]:
table.head()

,date,1,2,3,4,5,6,7,8,9,10,year-month,Mkt-RF,SMB,HML,RF,10-1
0,1965-06-30,0.060130,0.048714,0.045933,0.043589,0.036186,0.041811,0.046411,0.054371,0.063069,0.098875,1965-6,-0.0551,-0.0434,0.0059,0.0035,0.038744
1,1965-07-30,0.063075,0.053535,0.041568,0.033944,0.035139,0.038997,0.038621,0.057570,0.064483,0.096872,1965-7,0.0143,0.0089,0.0220,0.0031,0.033797
2,1965-08-31,0.072201,0.046245,0.032984,0.030696,0.029874,0.025043,0.038289,0.041052,0.048866,0.081152,1965-8,0.0273,0.0284,-0.0100,0.0033,0.008950
3,1965-09-30,0.065206,0.056510,0.031870,0.023073,0.029898,0.027071,0.029531,0.040143,0.047262,0.069693,1965-9,0.0286,0.0064,-0.0013,0.0031,0.004488
4,1965-10-29,0.024870,0.017834,0.014394,0.005298,0.008482,0.007039,0.007144,0.010225,0.008659,0.022142,1965-10,0.0260,0.0252,0.0156,0.0031,-0.002728


### CAPM

In [131]:
list_portfolio_rank=[1,2,3,4,5,6,7,8,9,10,"10-1"]

for i in list_portfolio_rank[0:]:
    
    table['avg_return_{}'.format(i)] = table[i] - table["RF"]
    X = table[["Mkt-RF"]].values
    X = sm.add_constant(X)
    y = table['avg_return_{}'.format(i)].values
    
    reg_mean=\
    sm.OLS(y,X).fit(cov_type='HAC',
                    cov_kwds={'maxlags': lagged},
                    use_t=True).params[0]

    reg_t_stat=\
    sm.OLS(y,X).fit(cov_type='HAC',
                    cov_kwds={'maxlags': lagged},
                    use_t=True).tvalues[0]
    
    print("portfolio: {}, alpha: {}, t stat: {}".format(i, reg_mean, reg_t_stat))

print("")
print("Even after incorporating the Fama and French 3-factor model, portfolios in higher deciles continue to exhibit positive and significant alphas.")

portfolio: 1, alpha: -0.0004525244519375614, t stat: -0.0978139392223286
portfolio: 2, alpha: 0.0032390419758945776, t stat: 0.777273673729779
portfolio: 3, alpha: 0.005258887926086735, t stat: 1.3563701866721092
portfolio: 4, alpha: 0.006851129925583562, t stat: 1.9018795893769267
portfolio: 5, alpha: 0.0063664266659840376, t stat: 1.83296370551204
portfolio: 6, alpha: 0.007766632014458812, t stat: 2.357643591516422
portfolio: 7, alpha: 0.008089709446555517, t stat: 2.5116372832329397
portfolio: 8, alpha: 0.008876532897945329, t stat: 2.662449003803706
portfolio: 9, alpha: 0.009558053642924327, t stat: 2.6287464677310615
portfolio: 10, alpha: 0.011407129128898076, t stat: 2.7156829652148495
portfolio: 10-1, alpha: 0.006098467553690771, t stat: 2.702921644395228

Even after incorporating the Fama and French 3-factor model, portfolios in higher deciles continue to exhibit positive and significant alphas.


### Fama-French three-factor alpha

In [130]:
list_portfolio_rank=[1,2,3,4,5,6,7,8,9,10,"10-1"]

for i in list_portfolio_rank[0:]:
    
    table['avg_return_{}'.format(i)] = table[i] - table["RF"]
    X = table[["Mkt-RF", "SMB", "HML"]].values
    X = sm.add_constant(X)
    y = table['avg_return_{}'.format(i)].values
    
    reg_mean=\
    sm.OLS(y,X).fit(cov_type='HAC',
                    cov_kwds={'maxlags': lagged},
                    use_t=True).params[0]

    reg_t_stat=\
    sm.OLS(y,X).fit(cov_type='HAC',
                    cov_kwds={'maxlags': lagged},
                    use_t=True).tvalues[0]
    
    print("portfolio: {}, alpha: {}, t stat: {}".format(i, reg_mean, reg_t_stat))

print("")
print("Even after incorporating the Fama and French 3-factor model, portfolios in higher deciles continue to exhibit positive and significant alphas.")

portfolio: 1, alpha: 0.000431665302473605, t stat: 0.09071084267714839
portfolio: 2, alpha: 0.004117018461485605, t stat: 0.9530285845196222
portfolio: 3, alpha: 0.005985640115007141, t stat: 1.4966118614804556
portfolio: 4, alpha: 0.0074754205324736945, t stat: 2.013420752471414
portfolio: 5, alpha: 0.006907565374424168, t stat: 1.947827885778176
portfolio: 6, alpha: 0.008259463504581979, t stat: 2.4504976842146045
portfolio: 7, alpha: 0.008525174785128441, t stat: 2.587680524425047
portfolio: 8, alpha: 0.009272253034692098, t stat: 2.7327622959517215
portfolio: 9, alpha: 0.009767924894861085, t stat: 2.6545250042108712
portfolio: 10, alpha: 0.011845642643813433, t stat: 2.7832388609475722
portfolio: 10-1, alpha: 0.005671247148183723, t stat: 2.434880345067214

Even after incorporating the Fama and French 3-factor model, portfolios in higher deciles continue to exhibit positive and significant alphas.
